In [61]:
import pandas as pd
from ast import literal_eval
import re

In [76]:
def extract_numbers(uid):
    match = re.search(r'_(\d+)_(\d+)', uid)
    if match:
        return int(match.group(1)), int(match.group(2))  # Erste und zweite Zahl zurückgeben
    return None, None

In [82]:
def get_rater_df(rater: int):
    rater = pd.read_csv(f"Text-Guided_IE_rater{rater}.tsv", sep="\t")
    rater = rater[[ "uid", "MagicBrush"]]
    rater[['id', 'turn']] = rater['uid'].apply(lambda x: pd.Series(extract_numbers(x)))
    rater["MagicBrush"] = rater["MagicBrush"].apply(literal_eval)
    rater["SC"] = rater["MagicBrush"].apply(lambda x: x[0])
    rater["PQ"] = rater["MagicBrush"].apply(lambda x: x[1])
    rater = rater.drop(["MagicBrush"], axis=1)
    rater = rater.drop(["uid"], axis=1)
    return rater

In [83]:
r1 = get_rater_df(1)

In [84]:
r1.head(5)

,id,turn,SC,PQ
0,100081,1,0.0,1.0
1,100081,3,0.5,0.5
2,100558,1,0.0,1.0
3,102171,1,0.5,0.5
4,102625,1,0.0,0.0


In [85]:
r1.to_csv("rater1.csv", index=False)

In [86]:
r2 = get_rater_df(2)

In [87]:
r2.to_csv("rater2.csv", index=False)

In [88]:
r3 = get_rater_df(3)

In [89]:
r3.to_csv("rater3.csv", index=False)

In [94]:
dev = pd.read_csv("/home/jovyan/BA/Github/MagicBrush/dev_data.csv", sep=",")
dev.head()

,img_id,turn_index,source_img,target_img,instruction
0,360871,1,/home/jovyan/BA/Github/MagicBrush/dev-00002-of...,/home/jovyan/BA/Github/MagicBrush/dev-00002-of...,What if the vegetables are in a bowl?
1,360871,2,/home/jovyan/BA/Github/MagicBrush/dev-00002-of...,/home/jovyan/BA/Github/MagicBrush/dev-00002-of...,Let's add a drawing of a flower to the fridge.
2,147745,1,/home/jovyan/BA/Github/MagicBrush/dev-00002-of...,/home/jovyan/BA/Github/MagicBrush/dev-00002-of...,Let the sun shine.
3,289514,1,/home/jovyan/BA/Github/MagicBrush/dev-00002-of...,/home/jovyan/BA/Github/MagicBrush/dev-00002-of...,It could be a microwave next to the woman.
4,289514,2,/home/jovyan/BA/Github/MagicBrush/dev-00002-of...,/home/jovyan/BA/Github/MagicBrush/dev-00002-of...,Can it be a boy not a girl?


In [95]:
found_turn=1
found_id=100081

In [96]:
source_img = dev[(dev["turn_index"] == found_turn) & (dev["img_id"] == found_id)]["source_img"].iloc[0]
target_img = dev[(dev["turn_index"] == found_turn) & (dev["img_id"] == found_id)]["target_img"].iloc[0]   

In [97]:
source_img

'/home/jovyan/BA/Github/MagicBrush/dev-00000-of-00004-f147d414270a90e1/100081-input.png'

In [2]:
import pandas as pd

# Beispiel-DataFrames
df1 = pd.read_csv("rater1.csv")
df2 = pd.read_csv("rater2.csv")
df3 = pd.read_csv("rater3.csv")

merged_df = df1.merge(df2, on=['turn', 'id'], suffixes=('_df1', '_df2'))
merged_df = merged_df.merge(df3, on=['turn', 'id'])

# Korrelationsmatrix berechnen für SC und PQ
correlation_sc = merged_df[['SC_df1', 'SC_df2', 'SC']].corr()
correlation_pq = merged_df[['PQ_df1', 'PQ_df2', 'PQ']].corr()

print("Korrelation SC:")
print(correlation_sc)

print("\nKorrelation PQ:")
print(correlation_pq)

Korrelation SC:
          SC_df1    SC_df2        SC
SC_df1  1.000000  0.731914  0.606169
SC_df2  0.731914  1.000000  0.649575
SC      0.606169  0.649575  1.000000

Korrelation PQ:
          PQ_df1    PQ_df2        PQ
PQ_df1  1.000000  0.605739  0.415906
PQ_df2  0.605739  1.000000  0.484380
PQ      0.415906  0.484380  1.000000
